In [1]:
import pandas as pd
cl = pd.read_csv('data/geolocated_filtered_listings.csv')
cl[:10]

,pid,date,region,neighborhood,rent,bedrooms,sqft,rent_sqft,rent_sqft_cat,longitude,latitude
0,4454264047,2014-05-11,santabarbara,NaN,3500.0,3.0,1200.0,2.916667,5,-119.726987,34.399757
1,4437398427,2014-05-11,santabarbara,6533 Trigo Road,850.0,2.0,882.0,0.963719,2,-119.855845,34.411019
2,4437286111,2014-05-11,santabarbara,downtown,2795.0,3.0,1200.0,2.329167,5,-119.712193,34.413942
3,4465598705,2014-05-11,santabarbara,santa barbara,5000.0,3.0,2000.0,2.500000,5,-119.692343,34.431691
4,4465561098,2014-05-11,santabarbara,Lompoc,1330.0,2.0,1000.0,1.330000,4,-120.469315,34.703429
5,4465459067,2014-05-11,santabarbara,256 Mathilda drive,1375.0,1.0,650.0,2.115385,5,-119.886525,34.428311
6,4465454916,2014-05-11,santabarbara,255 Ellwood Beach drive,1375.0,1.0,650.0,2.115385,5,-119.885960,34.428245
7,4428556398,2014-05-11,santabarbara,Santa Barbara,2400.0,2.0,850.0,2.823529,5,-119.669995,34.418522
8,4421044961,2014-05-11,santabarbara,vandenberg,850.0,NaN,420.0,2.023810,5,-120.468112,34.699481
9,4465408945,2014-05-11,santabarbara,"Lompoc, CA",850.0,1.0,644.0,1.319876,4,-120.461958,34.650227


In [6]:
import statsmodels.api as sm
import numpy as np
from patsy import dmatrices
y, X = dmatrices('np.log(rent) ~ np.log(sqft) + bedrooms', data=cl, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:           np.log(rent)   R-squared:                       0.212
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                 1.862e+05
Date:                Thu, 06 Oct 2016   Prob (F-statistic):               0.00
Time:                        15:40:08   Log-Likelihood:            -8.7752e+05
No. Observations:             1386600   AIC:                         1.755e+06
Df Residuals:                 1386597   BIC:                         1.755e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept        1.5083      0.011    139.498   

In [7]:
cl.shape

(1456338, 11)

In [47]:
sample = cl[:100]

In [23]:
import requests, json
def get_fips(row):
    url = 'http://data.fcc.gov/api/block/find?format=json&latitude={}&longitude={}'
    request = url.format(row['latitude'], row['longitude'])
    response = requests.get(request)
    data = response.json()
    return pd.Series({'fips_block':data['Block']['FIPS'], 'state':data['State']['code'], 'county':data['County']['name']})


fips = sample.apply(get_fips, axis=1)
sample = pd.concat([sample, fips], axis=1)

In [48]:
sample.shape[0]

100

In [49]:
start = 1
cl_geocoded = sample
for i in range(14563):
    sample = cl[start:start+100]
    fips = sample.apply(get_fips, axis=1)
    sample = pd.concat([sample, fips], axis=1) 
    cl_geocoded = pd.concat([cl_geocoded, sample])
    start = start+100
    pct = i/cl.shape[0]
    print('{:3.1f}'.format(pct), end=",")

TypeError: 'float' object cannot be interpreted as an integer

In [45]:
cl_geocoded.shape

(60, 14)

In [46]:
cl_geocoded[:60]

,pid,date,region,neighborhood,rent,bedrooms,sqft,rent_sqft,rent_sqft_cat,longitude,latitude,county,fips_block,state
1,4437398427,2014-05-11,santabarbara,6533 Trigo Road,850.0,2.0,882.0,0.963719,2,-119.855845,34.411019,Santa Barbara,060830029242004,CA
2,4437286111,2014-05-11,santabarbara,downtown,2795.0,3.0,1200.0,2.329167,5,-119.712193,34.413942,Santa Barbara,060830012031000,CA
3,4465598705,2014-05-11,santabarbara,santa barbara,5000.0,3.0,2000.0,2.500000,5,-119.692343,34.431691,Santa Barbara,060830006002000,CA
4,4465561098,2014-05-11,santabarbara,Lompoc,1330.0,2.0,1000.0,1.330000,4,-120.469315,34.703429,Santa Barbara,060830028084014,CA
5,4465459067,2014-05-11,santabarbara,256 Mathilda drive,1375.0,1.0,650.0,2.115385,5,-119.886525,34.428311,Santa Barbara,060830029304000,CA
6,4465454916,2014-05-11,santabarbara,255 Ellwood Beach drive,1375.0,1.0,650.0,2.115385,5,-119.885960,34.428245,Santa Barbara,060830029304000,CA
7,4428556398,2014-05-11,santabarbara,Santa Barbara,2400.0,2.0,850.0,2.823529,5,-119.669995,34.418522,Santa Barbara,060830008041005,CA
8,4421044961,2014-05-11,santabarbara,vandenberg,850.0,NaN,420.0,2.023810,5,-120.468112,34.699481,Santa Barbara,060830028084019,CA
9,4465408945,2014-05-11,santabarbara,"Lompoc, CA",850.0,1.0,644.0,1.319876,4,-120.461958,34.650227,Santa Barbara,060830027052010,CA
10,4450609529,2014-05-11,santabarbara,Storke Ranch,6000.0,4.0,2229.0,2.691790,5,-119.862235,34.418558,Santa Barbara,060830029225000,CA
